# Tutorial 05 - Comparison and `DataFrame` Masking

This tutorial begins with a discussion of a programming technique called *comparison*.

We then see how *comparison* can be used to select subsets of a `DataFrame` according to a variety of criteria.

Using comparison to subset a `DataFrame` is known as *masking*.

## Imporing Packages

Let's import the packages that we will neeed throughout this tutorial.

In [ ]:
##> import numpy as np
##> import pandas as pd
##> import datetime as dt



Notice that we are using the `datetime` package for the first time, which will help us to work with dates.

## Comparison

In data analysis, you often have occasion to compare two variables.

This means you are checking for conditions like equality (`==`), non-equality (`!=`), less-than (`<`).

The comparison of variables creates `boolean` data: `True` or `False`.

The following code code demonstrates comparisons with strings.

In [ ]:
##> str_x = "SPY"
##> str_y = "IWM"
##> str_z = "SPY"

##> str_x == str_y
##> str_x == str_z
##> str_x == str_y





This code demonstrates comparisons involving numbers.

In [ ]:
##> dbl_x = 287.50
##> dbl_y = 250

##> dbl_x > dbl_y 
##> dbl_x < dbl_y
##> dbl_x == dbl_y
##> dbl_x != dbl_y





You can also assign the boolean that is generated by comparison.

In [ ]:
##> bln_1 = (dbl_x == dbl_y)
##> bln_2 = (str_x == str_y)
##> bln_1
##> bln_2





## Boolean Operators

A quick aside on *boolean operators*...

A boolean operator takes in one or more booleans, and returns another boolean.

There are three operators to be aware of:

- `not` (unary) - takes a single boolean and returns it's negation

- `or` (binary) - takes two booleans and returns `True` if at least one is `True`

- `and` (binary) - takes two booleans and returns `True` only if both are `True`

Let's domonstrate the `not` operator:

In [ ]:
##> not True
##> not False




Let's demonstrate the `or` operator:

In [ ]:
##> True or True
##> True or False
##> False or True
##> False or False




And lastly, let's demonstrate the `and` operator:

In [ ]:
##> True and True
##> True and False
##> False and True
##> False and False





It is possible to combine these operators together is various ways, using parentheses, to come up with complext expressions:

In [ ]:
##> (not True) and (False and not(False or True))




This can seem a little confusing at first, but it always reduces down to the simple unary and binary boolean operators we discussed above.

## Reading in Sample Data

For the remainder of the discussion, we will need some data to work with.
Let's read in our sample SPY data 

In [ ]:
##> df_spy = pd.read_csv('../data/spy_dec_2018.csv')
##> df_spy




It's always helpful try to explore your data a little, once you have imported it.  One thing that I like to check for is the unique number of values in categorical columns, like `date`.

That is what is done in the following code:

In [ ]:
##> df_spy['date'].unique().size



The above calculation is not important to the rest of the tutorial, but I will be peppering little tips and tricks throughout these tutorials to facilitate learning via repetition.

## Comparison and DataFrame Columns

As discussed in a previous tutorial, a column of a `DataFrame` is a `Series` object, which basically just a glorfied `numpy.array` (think vector or matrix).

Let's separate out the `adjusted` column of `df_spy` and assign it to a variable:

In [ ]:
##> pd.options.display.max_rows = 6 # this modifys the printing of dataframes
##> ser_adjusted = df_spy['adjusted']
##> ser_adjusted




Recall that a `pandas.Series` is smart with respect to compnent-wise operations, meaning it behaves like vectors from linear algebra would.

This means that the following operations are *broadcasted* as you might expect.

In [ ]:
##> ser_adjusted - 100
##> ser_adjusted / 100




As it turns out, this broadcasting behavior also occurs when comparison.

The following code demonstrates this:

In [ ]:
##> ser_test = (ser_adjusted > 250)

##> ser_test
##> type(ser_test)
##> ser_test.values




Let's make a few observation about what just happened:

1. When we compare a `Series` of numerical values (`ser_adjusted`) to a single number (250), we get back a `Series` of booleans (`ser_test`)

2. We have that `ser_test[i]` = (`ser_adjusted[i] > 250`).

3. So the comparison operation was broadcast as you would expect.

This is probably easiest to see by appending `ser_test` to `df_spy` and the reprinting the dataframe.  That's what the following code does:

In [ ]:
##> pd.options.display.max_rows = 25
##> df_spy['test'] = ser_test
##> df_spy




Notice that starting on 12/19/2018, the adjusted price of SPY was below 250.

The broadcasting of comparison can be use as a powerful way to query subsets of data from a `DataFrame`; we consider this technique next.

## Masking with DataFrames

The next few steps might seem a little strange, but bear with me, I assure you this is leading somewhere.

From the code below we know that `df_spy` has 18 rows:

In [ ]:
##> df_spy.shape



The following code creates a list consisting of 18 booleans, all of them `False`:

In [ ]:
##> lst_bool = [False] * 18
##> lst_bool




Now, let's see what happens when we feed this list of `False` booleans into `df_spy` using square brackets.

In [ ]:
##> df_spy[lst_bool]




It may look like nothing has happened, but really what has been returned is a an empty `DataFrame`. (Exercise: try to demonstrate this to yourself with the `type()` function.) 

Next let's modify `lst_bool` slightly, by changing the first entry to `True`, and then feed it into `df_spy` again.

In [ ]:
##> lst_bool[0] = True # this changes the first entry to True
##> df_spy[lst_bool]
##> df_spy.head()




So what happened?  Notice that `df_spy[lst_bool]` returns a `DataFrame` consisting only of the first row of `df_spy`.

Let's modify lst_bool once again, by setting the second entry of `df_spy` to `True`, and then once again repeat our little experiment. 

In [ ]:
##> lst_bool[1] = True # this sets the second component to True
##> df_spy[lst_bool]
##> df_spy.head()




**Punchline:** What is returned by the code `df_spy[lst_bool]` will be a `DataFrame` consisting of all the rows corresponding to the `True` entries of `lst_bool`.

This is the notion of `DataFrame` *masking*.

## Querying a DataFrame

In a data analysis, it is often the case that you will want to isolate certain rows of a `DataFrame`, call it `df`. The rows you are interested can often be identified with by the condition that a comparison in involving one or more rows of `df` is `True`.  

We can achieve this kind of row isolation by combining the broadcasting of camparison over `DataFrame` columns, with the masking of `DataFrames`.

Let's consider a concrete example.  The following code reads in a slightly more complicated dataset.  It consists of EOD prices, for four different ETFs (SPY, IWM, QQQ, DIA), during the month of December 2018:

In [ ]:
##> pd.options.display.max_rows = 25 # changes displaying behavior 
##> df_etf = pd.read_csv("../data/index_etf_dec_2018.csv")
##> df_etf




As a first example of masking, let's isolate all the rows corresponding to `IWM`:

In [ ]:
##> pd.options.display.max_rows = 6
##> lst_bool = (df_etf['symbol'] == "IWM")
##> df_etf[lst_bool]




Notice that we did this in two steps: (1) calculate the list of `booleans`; (2) perform the masking by using square brackets `[]` and `lst_bool`.

We can actually perform this masking in a single step:

In [ ]:
##> df_etf[df_etf['symbol'] == "IWM"]




For our next example, let's isolate the price data from the December 2018 regular opion expiration, which was 12/21/2018.  In order to perform comparison on the `date` column, we will need to convert it to a `datetime` (it is currently an `object` data type).

In [ ]:
##> df_etf[df_etf['date'] == '2018-12-21']





The possibilites masking are endless, and it will be a staple of our data analysis toolbox.

## Related Reading

*PDSH* - 2.6 - Comparisons, Masks, and Boolean Logic

*PDSH* - 2.7 - Fancy Indexing

*PDSH* - 3.2 - Data Indexing and Selection 